## Scraping NBA Team Colors

In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import re

In [2]:
url = 'https://teamcolorcodes.com/nba-team-color-codes/'
html = urlopen(url)
soup = BeautifulSoup(html)

In [3]:
teams = soup.findAll('a', {"class": "team-button"})
teams[:5]

[<a class="team-button" href="https://teamcolorcodes.com/atlanta-hawks-color-codes/" style="background-color: #E03A3E; color: white; border-bottom: 4px solid #C1D32F;">Atlanta Hawks</a>,
 <a class="team-button" href="https://teamcolorcodes.com/boston-celtics-color-codes/" style="background-color: #007a33; color: white; border-bottom: 4px solid #BA9653;">Boston Celtics</a>,
 <a class="team-button" href="https://teamcolorcodes.com/brooklyn-nets-color-codes/" style="background-color: #000; color: white; border-bottom: 4px solid #000;">Brooklyn Nets</a>,
 <a class="team-button" href="https://teamcolorcodes.com/charlotte-hornets-color-codes/" style="background-color: #1D1160; color: white; border-bottom: 4px solid #00788c;">Charlotte Hornets</a>,
 <a class="team-button" href="https://teamcolorcodes.com/chicago-bulls-color-codes/" style="background-color: #CE1141; color: white; border-bottom: 4px solid #000;">Chicago Bulls</a>]

In [4]:
pattern = re.compile(r'background-color: (#.{3}|#.{6});.*border-bottom:.*solid (#.{3}|#.{6});')

colors = []
for a in teams:
    team_name = a.getText()
    color_list = list(re.findall(pattern, a['style'])[0])
    assert len(color_list) == 2
    for i, hex_code in enumerate(color_list):
        if len(hex_code) == 4:
            color_list[i] = f'#{"".join(2 * char for char in hex_code.lstrip("#"))}'
    colors.append([team_name, color_list[0].upper(), color_list[1].upper()])
    
colors[:5]

[['Atlanta Hawks', '#E03A3E', '#C1D32F'],
 ['Boston Celtics', '#007A33', '#BA9653'],
 ['Brooklyn Nets', '#000000', '#000000'],
 ['Charlotte Hornets', '#1D1160', '#00788C'],
 ['Chicago Bulls', '#CE1141', '#000000']]

In [5]:
df_colors = pd.DataFrame(colors, columns = ['Franchise', 'Color1', 'Color2'])
df_colors.loc[df_colors.Franchise == 'Seattle Supersonics', 'Franchise'] = 'Seattle SuperSonics'
df_colors.head()

,Franchise,Color1,Color2
0,Atlanta Hawks,#E03A3E,#C1D32F
1,Boston Celtics,#007A33,#BA9653
2,Brooklyn Nets,#000000,#000000
3,Charlotte Hornets,#1D1160,#00788C
4,Chicago Bulls,#CE1141,#000000


In [6]:
abb = pd.read_csv('../data/scraped/NBA_Teams_Abb.csv').rename(columns = {'Abbreviation/Acronym': 'Tm'})
abb.head()

,Tm,Franchise
0,ATL,Atlanta Hawks
1,BKN,Brooklyn Nets
2,BOS,Boston Celtics
3,CHA,Charlotte Hornets
4,CHI,Chicago Bulls


In [7]:
df = pd.merge(df_colors, abb, on = 'Franchise', how = 'left').set_index(['Franchise', 'Tm'])
df

,,Color1,Color2
Franchise,Tm,,
Atlanta Hawks,ATL,#E03A3E,#C1D32F
Boston Celtics,BOS,#007A33,#BA9653
Brooklyn Nets,BKN,#000000,#000000
Charlotte Hornets,CHA,#1D1160,#00788C
Chicago Bulls,CHI,#CE1141,#000000
Cleveland Cavaliers,CLE,#6F263D,#FFB81C
Dallas Mavericks,DAL,#00538C,#B8C4CA
Denver Nuggets,DEN,#00285E,#FFC72C
Detroit Pistons,DET,#ED174C,#006BB6


In [8]:
def bgColor(color):
    return f'background-color: { color }'

def textColor(color):
    r = int(color[1:3], 16)
    g = int(color[3:6], 16)
    b = int(color[6:9], 16)
    luma = 0.2126 * r + 0.7152 * g + 0.0722 * b
    return f'color: { "white" if luma < 1500 else "black"}'  

(df.style.
 applymap(bgColor, subset = ['Color1', 'Color2']).
 applymap(textColor, subset = ['Color1', 'Color2']))

,,Color1,Color2
Franchise,Tm,,
Atlanta Hawks,ATL,#E03A3E,#C1D32F
Boston Celtics,BOS,#007A33,#BA9653
Brooklyn Nets,BKN,#000000,#000000
Charlotte Hornets,CHA,#1D1160,#00788C
Chicago Bulls,CHI,#CE1141,#000000
Cleveland Cavaliers,CLE,#6F263D,#FFB81C
Dallas Mavericks,DAL,#00538C,#B8C4CA
Denver Nuggets,DEN,#00285E,#FFC72C
Detroit Pistons,DET,#ED174C,#006BB6


In [9]:
with open('../data/scraped/NBA_Teams_Color.csv', 'w') as f:
    f.write(df.to_csv())